This notebook demonstrates how to write a zarrtraj trajectory to disk and AWS S3 using the `ZarrTrajWriter`. This will serve as setup code for RMSF and RMSD
benchmarking notebooks as well.

Prerequisites: 
- `benchmark_setup.ipynb`

Steps:

1. Align the `.xtc` trajectory using `MDAnalysis.analysis.align`
2. Write the aligned trajectory to the local filesystem in `.xtc` format
3. Load the aligned trajectory into a universe and write it again in the local filesystem in `.zarrtraj` format
4. Write the loaded trajectory into an AWS S3 bucket

In [ ]:
from MDAnalysis.analysis import rms, align
import MDAnalysis as mda

# 1, 2

u = mda.Universe("notebook_data_tmp/yiip_equilibrium/YiiP_system.pdb",
                 "notebook_data_tmp/yiip_equilibrium/yiip.xtc")

average = align.AverageStructure(u, u, select="protein and name CA",
                                 ref_frame=0).run()
ref = average.results.universe
aligner = align.AlignTraj(u, ref,
                          select='protein and name CA',
                          filename='notebook_data_tmp/yiip_aligned.xtc',
                          in_memory=False).run()


In [1]:
import zarrtraj
import zarr
import MDAnalysis as mda

# 3

u = mda.Universe("notebook_data_tmp/yiip_equilibrium/YiiP_system.pdb",
                 "notebook_data_tmp/yiip_aligned.xtc")

zHDD = zarr.open_group("notebook_data_tmp/yiip_aligned.zarrtraj", 'w')

with mda.Writer(zHDD, u.atoms.n_atoms,
                format='ZARRTRAJ') as W:
                    for ts in u.trajectory:
                        W.write(u.atoms)

importing zarrtraj...


/nfs/homes3/ljwoods2/.conda/envs/zarrtraj/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:348: UserWarning: Unknown element Z found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/nfs/homes3/ljwoods2/.conda/envs/zarrtraj/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:348: UserWarning: Unknown element D found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/nfs/homes3/ljwoods2/.conda/envs/zarrtraj/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/nfs/homes3/ljwoods2/.conda/envs/zarrtraj/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to gu

In [ ]:
import os 
import s3fs
import zarrtraj
import zarr
import MDAnalysis as mda

# 4
# Must set the environmental variable using Jupyter's
# "%env KEY=VALUE"
# before running

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("BUCKET_NAME")

u = mda.Universe("notebook_data_tmp/yiip_equilibrium/YiiP_system.pdb",
                 "notebook_data_tmp/yiip_aligned.xtc")

s3_fs = s3fs.S3FileSystem(
    # anon must be false to allow authentication
    anon=False,
    client_kwargs=dict(
        region_name='us-east-1',
    )
)

cloud_store = s3fs.S3Map(
    root=f'{BUCKET_NAME}/yiip_aligned.zarrtraj',
    s3=s3_fs,
    check=False
)

zS3 = zarr.open_group(cloud_store, 'w')

with mda.Writer(zS3, u.atoms.n_atoms,
                n_frames=u.trajectory.n_frames,
                format='ZARRTRAJ') as W:
                    for ts in u.trajectory:
                        W.write(u.atoms)